# Practica 1

### 1a. Hacer un programa que cargue una imagen y que obtenga y muestre el valor RGB de cada píxel de la imagen sobre el que esté el cursor del ratón.

In [2]:
import cv2 as cv
import numpy as np

# Callback del ratón

def mostrar_rgb(event, x, y, flags, param):
    if event == cv.EVENT_MOUSEMOVE:
        b, g, r = img[y, x]
        texto = f"Pos ({x}, {y}) -> RGB ({r}, {g}, {b})"
        
        # Crear fondo negro
        info = np.zeros((60, 480, 3), np.uint8)
        cv.putText(info, texto, (10, 40), cv.FONT_HERSHEY_SIMPLEX, 
                   0.7, (0, 255, 255), 2, cv.LINE_AA)
        cv.imshow('Info', info)   


# Cargar imagen
img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

# Crear ventana y asociar el callback
cv.namedWindow('Imagen', cv.WINDOW_AUTOSIZE)
cv.setMouseCallback('Imagen', mostrar_rgb)
cv.imshow('Imagen', img)

# Bucle hasta pulsar ESC o 'q'
while True:
    key = cv.waitKey(1) & 0xFF
    if key == 27 or key == ord('q'):
        break

# Cierre robusto
cv.destroyWindow('Imagen')
cv.destroyWindow('Info')
cv.waitKey(1)

-1

### 1b. Hacer un programa para dibujar las siguientes primitivas con el ratón

- Líneas
- Rectángulos
- Círculos
- Especificar el color en RGB de las primitivas
- Definir el grosor de las líneas

In [9]:
import cv2
import numpy as np

drawing = False  
ix, iy = -1, -1  
mode = 'linea'    # modos: 'linea', 'rectangulo', 'circulo'


def nothing(x):
    pass

def draw(event, x, y, flags, param):
    global ix, iy, drawing, mode, img

    line_gross = cv2.getTrackbarPos('Gross', 'Dibujo')
    r = cv2.getTrackbarPos("R", "Dibujo")
    g = cv2.getTrackbarPos("G", "Dibujo")
    b = cv2.getTrackbarPos("B", "Dibujo")
    color = (b, g, r)

    if line_gross == 0:
        return

    if mode in ['linea', 'rectangulo', 'circulo']:
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            ix, iy = x, y

        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            temp = img.copy()
            if mode == 'linea':
                cv2.line(temp, (ix, iy), (x, y), color, line_gross)
            elif mode == 'rectangulo':
                cv2.rectangle(temp, (ix, iy), (x, y), color, line_gross)
            elif mode == 'circulo':
                radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
                cv2.circle(temp, (ix, iy), radius, color, line_gross)
            cv2.imshow("Dibujo", temp)

        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            if mode == 'linea':
                cv2.line(img, (ix, iy), (x, y), color, line_gross)
            elif mode == 'rectangulo':
                cv2.rectangle(img, (ix, iy), (x, y), color, line_gross)
            elif mode == 'circulo':
                radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
                cv2.circle(img, (ix, iy), radius, color, line_gross)
            cv2.imshow("Dibujo", img)

# Imagen inicial
img = np.ones((400, 400, 3), np.uint8) * 255

cv2.namedWindow("Dibujo")
cv2.setMouseCallback("Dibujo", draw)

cv2.createTrackbar('Gross', 'Dibujo', 0, 20, lambda x: None)
cv2.createTrackbar('R', 'Dibujo', 0, 255, nothing)
cv2.createTrackbar('G', 'Dibujo', 0, 255, nothing)
cv2.createTrackbar('B', 'Dibujo', 0, 255, nothing)

print("Controles:")
print("  Tecla 'l' -> Línea")
print("  Tecla 'r' -> Rectángulo")
print("  Tecla 'c' -> Círculo")

print("  Tecla 'ESC' -> Salir")

while True:
    cv2.imshow("Dibujo", img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  
        break
    elif k == ord('l'):
        mode = 'linea'; print("Modo: Línea")
    elif k == ord('r'):
        mode = 'rectangulo'; print("Modo: Rectángulo")
    elif k == ord('c'):
        mode = 'circulo'; print("Modo: Círculo")

cv2.destroyAllWindows()
cv2.waitKey(1)


Controles:
  Tecla 'l' -> Línea
  Tecla 'r' -> Rectángulo
  Tecla 'c' -> Círculo
  Tecla 'ESC' -> Salir
Modo: Círculo
Modo: Rectángulo


-1

### 2. Mejoras del programa:
- Permitir crear figuras rellenas (seleccionar a través de la interfaz).
- Poder crear otras primitivas (construir polilíneas y polígonos, elipses…)
- Guardar el dibujo como un vídeo para ver cómo se realizó paso a paso

In [ ]:
import cv2
import numpy as np

drawing = False  
ix, iy = -1, -1  
mode = 'linea'    # modos: 'linea', 'rectangulo', 'circulo', 'elipse', 'polilinea', 'poligono'
pts = []          # lista de puntos para polilineas/poligonos

def nothing(x):
    pass

def draw(event, x, y, flags, param):
    global ix, iy, drawing, mode, img, pts, out

    line_gross = cv2.getTrackbarPos('Gross', 'Dibujo')
    relleno = cv2.getTrackbarPos('rellena', 'Dibujo')
    r = cv2.getTrackbarPos("R", "Dibujo")
    g = cv2.getTrackbarPos("G", "Dibujo")
    b = cv2.getTrackbarPos("B", "Dibujo")
    color = (b, g, r)

    if relleno == 1 and mode in ['rectangulo', 'circulo', 'elipse', 'poligono']:
        line_gross = -1
    elif line_gross == 0:
        return


    if mode in ['linea', 'rectangulo', 'circulo', 'elipse']:
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            ix, iy = x, y
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            if mode == 'linea':
                cv2.line(img, (ix, iy), (x, y), color, line_gross)
            elif mode == 'rectangulo':
                cv2.rectangle(img, (ix, iy), (x, y), color, line_gross)
            elif mode == 'circulo':
                radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
                cv2.circle(img, (ix, iy), radius, color, line_gross)
            elif mode == 'elipse':
                axisX = abs(x - ix)
                axisY = abs(y - iy)
                cv2.ellipse(img, (ix, iy), (axisX, axisY), 0, 0, 360, color, line_gross)
            cv2.imshow("Dibujo", img)
            out.write(img)    # <<< guardamos el definitivo
            
    # --- Polilineas / Poligonos ---
    elif mode in ['polilinea', 'poligono']:
        if event == cv2.EVENT_LBUTTONDOWN:
            pts.append([x, y])  # añade punto
            cv2.circle(img, (x, y), 2, (0, 0, 0), -1)
            if len(pts) > 1:
                cv2.line(img, tuple(pts[-2]), tuple(pts[-1]), color, 1)
            out.write(img)

        elif event == cv2.EVENT_RBUTTONDOWN:  # clic derecho para cerrar
            if len(pts) > 2:
                pts_np = np.array(pts, np.int32).reshape((-1, 1, 2))
                if mode == 'polilinea':
                    cv2.polylines(img, [pts_np], False, color, line_gross)
                elif mode == 'poligono':
                    if relleno == 1:
                        cv2.fillPoly(img, [pts_np], color)
                    else:
                        cv2.polylines(img, [pts_np], True, color, line_gross)
            pts = []  # resetea puntos
            out.write(img)


# Imagen inicial
img = np.ones((400, 400, 3), np.uint8) * 255

cv2.namedWindow("Dibujo")
cv2.setMouseCallback("Dibujo", draw)

cv2.createTrackbar('Gross', 'Dibujo', 0, 20, lambda x: None)
cv2.createTrackbar('R', 'Dibujo', 0, 255, nothing)
cv2.createTrackbar('G', 'Dibujo', 0, 255, nothing)
cv2.createTrackbar('B', 'Dibujo', 0, 255, nothing)
cv2.createTrackbar('rellena', 'Dibujo', 0, 1, nothing)

# Configuración del vídeo MP4
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('dibujo.mp4', fourcc, 10, (400, 400))

print("Controles:")
print("  Tecla 'l' -> Línea")
print("  Tecla 'r' -> Rectángulo")
print("  Tecla 'c' -> Círculo")
print("  Tecla 'e' -> Elipse")
print("  Tecla 'p' -> Polilínea (clics sucesivos, clic derecho para cerrar)")
print("  Tecla 'o' -> Polígono (clics sucesivos, clic derecho para cerrar)")
print("  Tecla 'ESC' -> Salir")

while True:
    cv2.imshow("Dibujo", img)
    out.write(img)   # <<< guardamos el estado actual de la imagen
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  
        break
    elif k == ord('l'):
        mode = 'linea'; print("Modo: Línea")
    elif k == ord('r'):
        mode = 'rectangulo'; print("Modo: Rectángulo")
    elif k == ord('c'):
        mode = 'circulo'; print("Modo: Círculo")
    elif k == ord('e'):
        mode = 'elipse'; print("Modo: Elipse")
    elif k == ord('p'):
        mode = 'polilinea'; pts = []; print("Modo: Polilínea")
    elif k == ord('o'):
        mode = 'poligono'; pts = []; print("Modo: Polígono")

out.release()   # cerramos el archivo de vídeo
cv2.destroyAllWindows()
cv2.waitKey(1)


Controles:
  Tecla 'l' -> Línea
  Tecla 'r' -> Rectángulo
  Tecla 'c' -> Círculo
  Tecla 'e' -> Elipse
  Tecla 'p' -> Polilínea (clics sucesivos, clic derecho para cerrar)
  Tecla 'o' -> Polígono (clics sucesivos, clic derecho para cerrar)
  Tecla 'ESC' -> Salir
Modo: Círculo


-1